In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors, gridspec
import numpy as np
import os

In [2]:
def define_x_countries_have_data(df, pop, timevar, mint, maxt, origin_fe=None, by_country=False):
    
    iso_codes = ['BEN', 'BFA', 'BWA', 'CMR', 'EGY', 
                 'ETH', 'GHA', 'GIN', 'LBR', 'MLI', 
                 'MOZ', 'MUS', 'MWI', 'RWA', 'SEN', 
                 'SLE', 'TGO', 'UGA', 'ZAF', 'ZMB']
    
    iso_df = pd.DataFrame({'aux': 1}, index=sorted(iso_codes))
    religs = ['Christian', 'Muslim', 'Traditional']
    
    if timevar == 'age':
        x = (df[df.bd==1980][['iso', timevar, 'year', 'major_religion', 'org', 'migrant', 'w']]
             .groupby(['iso', timevar, 'year', 'major_religion', 'org', 'migrant'])
             .sum()
             .reset_index(drop=False))
    else:
        x = (df[['iso', timevar, 'year', 'major_religion', 'org', 'migrant', 'w']]
             .groupby(['iso', timevar, 'year', 'major_religion', 'org', 'migrant'])
             .sum()
             .reset_index(drop=False))

    x = x.set_index(['iso', timevar, 'year', 'major_religion', 'org', 'migrant'], drop=True).unstack().fillna(0).reset_index(drop=False)
    x.columns = ['iso', timevar, 'year', 'major_religion', 'org', 'w0', 'w1']
    x['tot'] = x.w0 + x.w1
    x['shr'] = x.w1 / x.tot
    x.drop(['w0', 'w1'], axis=1, inplace=True)

    if origin_fe == 'unweighted':
        x['total_org_migshr'] = x.groupby(['iso', timevar, 'year', 'org'])['shr'].transform('mean')
        x['shr'] = x.shr - x.total_org_migshr        
        x.drop('total_org_migshr', axis=1, inplace=True)
    elif origin_fe == 'weighted':
        x['tottot'] = x.groupby(['iso', timevar, 'year', 'org'])['tot'].transform('sum')
        x['orgobshr'] = x.tot / x.tottot
        x['wshr'] = x.shr * x.orgobshr
        x['total_org_migshr'] = x.groupby(['iso', timevar, 'year', 'org'])['wshr'].transform('sum')

        # x['aux'] = x.groupby(['iso', timevar, 'year', 'org'])['orgobshr'].transform('sum')
        # print(x[x.major_religion.isin(religs)].aux.min(), x[x.major_religion.isin(religs)].aux.max())
        # x.drop('aux', axis=1)

        x['shr'] = x.shr - x.total_org_migshr
        x.drop(['tottot', 'orgobshr', 'wshr', 'total_org_migshr'], axis=1, inplace=True)
    
    x['tottot'] = x.groupby(['iso', timevar, 'year', 'major_religion'])['tot'].transform('sum')
    x['orgshr'] = x.tot / x.tottot
    x['wshr'] = x.shr * x.orgshr

    # x['aux'] = x.groupby(['iso', timevar, 'year', 'major_religion'])['orgshr'].transform('sum')
    # print(x[x.major_religion.isin(religs)].aux.min(), x[x.major_religion.isin(religs)].aux.max())
    # x.drop('aux', axis=1)

    x = x.groupby(['iso', timevar, 'year',  'major_religion'])[['wshr']].sum().reset_index(drop=False)
    x = x.groupby(['iso', timevar, 'major_religion'])[['wshr']].mean().reset_index(drop=False)
    
    if by_country:
        return x
    else:
        y = x[(x[timevar]>=mint) & (x[timevar]<=maxt)].reset_index(drop=True).copy()
        countries_have_data = {}
        for relig in religs:
            yi = (y[(y.major_religion==relig)]
                   .drop(['major_religion'], axis=1)
                   .groupby(['iso', timevar]).sum()
                   .unstack()
                   )
            yi['wshr'] = np.abs(yi.wshr)
            yi = yi.fillna(0)
            yi[yi>0] = 1
            yi.columns = yi.columns.droplevel(0)
            yi.columns = [f'{c}' for c in yi.columns]
            countries_have_data[relig] = yi.join(iso_df, how='outer').drop('aux', axis=1).fillna(0)

        if timevar == 'bd':
            x['bdx'] = x.bd
            x.loc[x.bd<1950, 'bd'] = 1950
            x.loc[x.bd>2000, 'bd'] = 2000
            x = pd.merge(x, pop, on=['iso', 'bd'], how='left')
            x['bd'] = x.bdx
            x.drop('bdx', axis=1, inplace=True)
        else:
            x = pd.merge(x, pop[pop.bd==1980].drop('bd', axis=1), on=['iso'], how='left')

        # normalize the population share for each birth decade X major religion
        x['aux'] = x.groupby([timevar, 'major_religion'])['popshr'].transform('sum')
        x['popshr'] = x.popshr / x.aux
        # x['aux'] = x.groupby([timevar, 'major_religion'])['popshr'].transform('sum')
        # print(x[x.major_religion.isin(religs)].aux.min(), x[x.major_religion.isin(religs)].aux.max())
        x['wshr'] = x.wshr * x.popshr
        x.drop(['iso', 'popshr', 'aux'], axis=1, inplace=True)
        x = x.groupby([timevar, 'major_religion']).sum().reset_index(drop=False)
        x = x[(x[timevar]>=mint) & (x[timevar]<=maxt)].reset_index(drop=True)

        return x, countries_have_data


def make_plots(x, countries_have_data, timevar, mint, maxt):
    
    religs = ['Christian', 'Muslim', 'Traditional']
    linecolors = ['r', 'b', 'k']
    linestyles = ['-', '--', ':']
    
    f = plt.figure(figsize=(20,10))
    ax = []
    ax.append(f.add_subplot(2,2,1))
    ax.append(f.add_subplot(3,2,2))
    ax.append(f.add_subplot(3,2,4))
    ax.append(f.add_subplot(3,2,6))
        
    for i in range(3):
    
        ax[0].plot(x[(x.major_religion==religs[i])][timevar], 
                   x[(x.major_religion==religs[i])].wshr,
                   color=linecolors[i],
                   linestyle=linestyles[i],
                   label=religs[i]
                  )
        
        ax[0].grid(color='lightgray')
        ax[0].legend()
        if timevar == 'bd':
            ax[0].set_xlabel('birth decade')
        else:
            ax[0].set_xlabel('age')
        ax[0].set_ylabel('migrant share')
        
        ax[i+1].imshow(countries_have_data[religs[i]], cmap=colors.ListedColormap(['grey', 'white']))

        
        if timevar == 'bd':
            ax[i+1].set_aspect(0.1)
            xtlabs = np.arange(mint, maxt+10, 10)
            ax[i+1].set_xticks(np.arange(len(xtlabs)))
            ax[i+1].set_xticklabels(xtlabs)
            ax[i+1].hlines(y=np.arange(0, 20)+0.5, xmin=np.full(20, 0)-0.5, xmax=np.full(20, len(xtlabs))-0.5, color="k")
            ax[i+1].vlines(x=np.arange(0, len(xtlabs))+0.5, ymin=np.full(len(xtlabs), 0)-0.5, ymax=np.full(len(xtlabs), 20)-0.5, color="black")
        else:
            ax[i+1].set_aspect(0.35)
            xtlabs = np.arange(mint, maxt+1, 1)
            ax[i+1].set_xticks(np.arange(len(xtlabs)))
            ax[i+1].set_xticklabels(xtlabs)
            ax[i+1].hlines(y=np.arange(0, 20)+0.5, xmin=np.full(20, 0)-0.5, xmax=np.full(20, len(xtlabs))-0.5, color="k")
            ax[i+1].vlines(x=np.arange(0, len(xtlabs))+0.5, ymin=np.full(len(xtlabs), 0)-0.5, ymax=np.full(len(xtlabs), 20)-0.5, color="black")
        ax[i+1].set_yticks(np.arange(20))
        ax[i+1].set_yticklabels(countries_have_data[religs[i]].index)
        ax[i+1].set_title('data availability by country: ' + religs[i])
        
        
    plt.close(f)
    return f

def bar_plot(df, leg_loc):
    
    religs = ['Christian', 'Muslim', 'Traditional']
    df = df[(df.bd==1980) & (df.major_religion.isin(religs))].reset_index(drop=True).drop('bd', axis=1)
    df = df.set_index(['iso', 'major_religion']).unstack().reset_index(drop=False)
    df.columns = ['iso', 'Christian', 'Muslim', 'Traditional']
    df = df.sort_values(['iso'], ascending=False).reset_index(drop=True)
    
    labels = list(df.iso)
    
    bar_width = 0.3
    x = np.arange(len(labels))
    f, ax = plt.subplots(figsize=(6, 10))
    ax.barh(x + bar_width, df.Christian, bar_width, label='Christian', color='r')
    ax.barh(x, df.Muslim, bar_width, label='Muslim', color='b')
    ax.barh(x - bar_width, df.Traditional, bar_width, label='Traditional', color='k') 
            
    ax.set_yticks(x)
    ax.set_yticklabels(labels)
    ax.legend(loc=leg_loc, prop={'size': 12}, bbox_to_anchor=(-0.03,-0.11), ncol=3)
    ax.tick_params(axis='both', labelsize=12)
    ax.set_ylim(x[0]-0.5, x[-1]+0.5)
    ax.grid(color='lightgray')
    ax.set_axisbelow(True)
    plt.close(f)
    return f

In [3]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
wdir = os.path.join(rootdir, '_2_intermediate', 'data')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [4]:
iso_codes = ['BEN', 'BFA', 'BWA', 'CMR', 'EGY', 
             'ETH', 'GHA', 'GIN', 'LBR', 'MLI', 
             'MOZ', 'MUS', 'MWI', 'RWA', 'SEN', 
             'SLE', 'TGO', 'UGA', 'ZAF', 'ZMB']

pop = pd.read_csv(os.path.join(wdir, 'pop_world.csv'))

pop = pop[pop.iso.isin(iso_codes)].reset_index(drop=True)
pop['tot'] = pop.groupby('year')['pop'].transform('sum')
pop['popshr'] = pop['pop'] / pop['tot']
pop['bd'] = pop.year - 18
pop['rem'] = pop.bd % 10
pop = pop[pop.rem==0].reset_index(drop=True)[['iso', 'bd', 'popshr']]

# Preliminary analysis: migrant shares by country-year
This is needed so we can check if there are definitional differences, or if the units defining migrants matter

In [5]:
df = pd.read_csv(os.path.join(wdir, 'migrant_stock_data_all.csv'))
df = df[['iso', 'year', 'migrant', 'w']].groupby(['iso', 'year', 'migrant']).sum().reset_index(drop=False)
df['tot'] = df.groupby(['iso', 'year'])['w'].transform('sum')
df['shr'] = df.w / df.tot
df = df.drop(['w', 'tot'], axis=1).set_index(['iso', 'year', 'migrant'], drop=True).unstack().reset_index(drop=False)
df.columns = ['iso', 'year', 'shr_nonmig', 'shr_mig']
df.drop('shr_nonmig', axis=1, inplace=True)

In [6]:
# df

- absolute outliers:
    - RWA 1991, ETH 1984, SEN 1988, ZAF 2001, MWI 2008
- relative outliers:
    - RWA 1991, SEN 1988, ZAF 2001, UGA 1991
    
May wish to discard them in robustness

In [7]:
outdir = '../data'

In [8]:
df.columns = ['iso code', 'census year', 'migrant share']
textab = df.to_latex(index=False, float_format="%.3f", caption="migrant shares")
textab = textab.replace('toprule', 'hline').replace('midrule', 'hline').replace('bottomrule', 'hline').replace('\\begin{table}', '\\begin{table}[ht!]')
with open(outdir + '/basic_migrant_share_by_census_table.tex', 'w') as fh:
    fh.write(textab)

# All observations

## by birth decade

In [9]:
df = pd.read_csv(os.path.join(wdir, 'migrant_stock_data_all.csv'))

In [10]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe=None)
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_none.pdf', bbox_inches='tight')

In [11]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_unweighted.pdf', bbox_inches='tight')

In [12]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_weighted.pdf', bbox_inches='tight')

## by age

In [13]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe=None)
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_none.pdf', bbox_inches='tight')

In [14]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_unweighted.pdf', bbox_inches='tight')

In [15]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_weighted.pdf', bbox_inches='tight')

# Robustness, excluding outlier country-years

In [16]:
df.shape

(708731, 8)

In [17]:
df['exclude'] = 0
df.loc[(df.iso == 'RWA') & (df.year == 1991), 'exclude'] = 1
df.loc[(df.iso == 'ETH') & (df.year == 1984), 'exclude'] = 1
df.loc[(df.iso == 'SEN') & (df.year == 1988), 'exclude'] = 1
df.loc[(df.iso == 'ZAF') & (df.year == 2001), 'exclude'] = 1
df.loc[(df.iso == 'MWI') & (df.year == 2008), 'exclude'] = 1
df.loc[(df.iso == 'UGA') & (df.year == 1991), 'exclude'] = 1
df = df[df.exclude == 0].reset_index(drop=True)

In [18]:
df.shape

(641340, 9)

In [19]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe=None)
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_none_nooutliers.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_unweighted_nooutliers.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_weighted_nooutliers.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe=None)
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_none_nooutliers.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_unweighted_nooutliers.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_weighted_nooutliers.pdf', bbox_inches='tight')

# By educational attainment

In [20]:
df = pd.read_csv(os.path.join(wdir, 'migrant_stock_data_ed.csv'))
df = df[df.lit0==0].reset_index(drop=True)

## All kids

In [21]:
x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe=None)
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_none_edall.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_unweighted_edall.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_weighted_edall.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe=None)
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_none_edall.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_unweighted_edall.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df, pop, 'age', 14, 30, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_weighted_edall.pdf', bbox_inches='tight')

## Educated kids

In [22]:
x, countries_have_data = define_x_countries_have_data(df[df.lit==1], pop, 'bd', 1950, 2010, origin_fe=None)
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_none_ed1.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==1], pop, 'bd', 1950, 2010, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_unweighted_ed1.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==1], pop, 'bd', 1950, 2010, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_weighted_ed1.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==1], pop, 'age', 14, 30, origin_fe=None)
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_none_ed1.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==1], pop, 'age', 14, 30, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_unweighted_ed1.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==1], pop, 'age', 14, 30, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_weighted_ed1.pdf', bbox_inches='tight')

## Uneducated kids

In [23]:
x, countries_have_data = define_x_countries_have_data(df[df.lit==0], pop, 'bd', 1950, 2010, origin_fe=None)
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_none_ed0.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==0], pop, 'bd', 1950, 2010, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_unweighted_ed0.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==0], pop, 'bd', 1950, 2010, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'bd', 1950, 2010)
f.savefig(outdir + '/migshares_bd_orgfe_weighted_ed0.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==0], pop, 'age', 14, 30, origin_fe=None)
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_none_ed0.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==0], pop, 'age', 14, 30, origin_fe='unweighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_unweighted_ed0.pdf', bbox_inches='tight')

x, countries_have_data = define_x_countries_have_data(df[df.lit==0], pop, 'age', 14, 30, origin_fe='weighted')
f = make_plots(x, countries_have_data, 'age', 14, 30)
f.savefig(outdir + '/migshares_age_orgfe_weighted_ed0.pdf', bbox_inches='tight')

# By-country results

## All observations

In [24]:
df = pd.read_csv(os.path.join(wdir, 'migrant_stock_data_all.csv'))

In [25]:
x = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe=None, by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_none.pdf', bbox_inches='tight')

In [26]:
x = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe="unweighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_unweighted.pdf', bbox_inches='tight')

In [27]:
x = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe="weighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_weighted.pdf', bbox_inches='tight')

## Kids of illiterate old

In [28]:
df = pd.read_csv(os.path.join(wdir, 'migrant_stock_data_ed.csv'))
df = df[df.lit0==0].reset_index(drop=True)

### All kids

In [29]:
x = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe=None, by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_none_edall.pdf', bbox_inches='tight')

x = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe="unweighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_unweighted_edall.pdf', bbox_inches='tight')

x = define_x_countries_have_data(df, pop, 'bd', 1950, 2010, origin_fe="weighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_weighted_edall.pdf', bbox_inches='tight')

### Educated kids

In [30]:
x = define_x_countries_have_data(df[df.lit==1], pop, 'bd', 1950, 2010, origin_fe=None, by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_none_ed1.pdf', bbox_inches='tight')

x = define_x_countries_have_data(df[df.lit==1], pop, 'bd', 1950, 2010, origin_fe="unweighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_unweighted_ed1.pdf', bbox_inches='tight')

x = define_x_countries_have_data(df[df.lit==1], pop, 'bd', 1950, 2010, origin_fe="weighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_weighted_ed1.pdf', bbox_inches='tight')

### Uneducated kids

In [31]:
x = define_x_countries_have_data(df[df.lit==0], pop, 'bd', 1950, 2010, origin_fe=None, by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_none_ed0.pdf', bbox_inches='tight')

x = define_x_countries_have_data(df[df.lit==0], pop, 'bd', 1950, 2010, origin_fe="unweighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_unweighted_ed0.pdf', bbox_inches='tight')

x = define_x_countries_have_data(df[df.lit==0], pop, 'bd', 1950, 2010, origin_fe="weighted", by_country=True)
f = bar_plot(x, 'lower left')
f.savefig(outdir + '/migshares_byc_bd1980_orgfe_weighted_ed0.pdf', bbox_inches='tight')

# Walkthrough of the function logic

In [32]:
timevar = 'bd'
mint = 1950
maxt = 2010
iso_df = pd.DataFrame({'aux': 1}, index=sorted(list(set(df.iso))))
religs = ['Christian', 'Muslim', 'Traditional']
origin_fe = 'weighted'
# origin_fe = None

In [33]:
df = pd.read_csv(os.path.join(wdir, 'migrant_stock_data_all.csv'))

In [34]:
df

,iso,year,bd,age,major_religion,org,migrant,w
0,BEN,1992,1890,93,Christian,33,0,10.0
1,BEN,1992,1890,93,Christian,53,0,10.0
2,BEN,1992,1890,93,Christian,76,0,10.0
3,BEN,1992,1890,93,Muslim,1,1,10.0
4,BEN,1992,1890,93,Muslim,3,0,20.0
...,...,...,...,...,...,...,...,...
708726,MUS,2011,2010,1,Christian,740,0,10.0
708727,MUS,2011,2010,1,Muslim,726,0,10.0
708728,MUS,2011,2010,1,Muslim,733,0,10.0
708729,MUS,2011,2010,1,Other,716,0,10.0


### 1. Summing by time variable
In the case of `timevar = "bd"`, this drops `age` and sums the `w` over all ages.

In [35]:
if timevar == 'age':
    x = (df[df.bd==1980][['iso', timevar, 'year', 'major_religion', 'org', 'migrant', 'w']]
         .groupby(['iso', timevar, 'year', 'major_religion', 'org', 'migrant'])
         .sum()
         .reset_index(drop=False))
else:
    x = (df[['iso', timevar, 'year', 'major_religion', 'org', 'migrant', 'w']]
         .groupby(['iso', timevar, 'year', 'major_religion', 'org', 'migrant'])
         .sum()
         .reset_index(drop=False))

In [36]:
x

,iso,bd,year,major_religion,org,migrant,w
0,BEN,1890,1992,Christian,1,0,30.0
1,BEN,1890,1992,Christian,6,0,10.0
2,BEN,1890,1992,Christian,8,0,10.0
3,BEN,1890,1992,Christian,10,0,20.0
4,BEN,1890,1992,Christian,11,0,50.0
...,...,...,...,...,...,...,...
112462,ZMB,2010,2010,Traditional,1234,0,50.0
112463,ZMB,2010,2010,Traditional,1235,0,80.0
112464,ZMB,2010,2010,Traditional,1236,0,50.0
112465,ZMB,2010,2010,Traditional,1237,0,60.0


### 2. Reshaping wide by migrant, computing migrant share and total observations

Note that `tot` contains the total observation weight in that `iso` X `timevar` X `year` X `major_religion` X `org` cell and `shr` is the migrant share

In [37]:
x = x.set_index(['iso', timevar, 'year', 'major_religion', 'org', 'migrant'], drop=True).unstack().fillna(0).reset_index(drop=False)
x.columns = ['iso', timevar, 'year', 'major_religion', 'org', 'w0', 'w1']
x['tot'] = x.w0 + x.w1
x['shr'] = x.w1 / x.tot
x.drop(['w0', 'w1'], axis=1, inplace=True)

In [38]:
x

,iso,bd,year,major_religion,org,tot,shr
0,BEN,1890,1992,Christian,1,30.0,0.0
1,BEN,1890,1992,Christian,6,10.0,0.0
2,BEN,1890,1992,Christian,8,10.0,0.0
3,BEN,1890,1992,Christian,10,20.0,0.0
4,BEN,1890,1992,Christian,11,50.0,0.0
...,...,...,...,...,...,...,...
63813,ZMB,2010,2010,Traditional,1234,50.0,0.0
63814,ZMB,2010,2010,Traditional,1235,80.0,0.0
63815,ZMB,2010,2010,Traditional,1236,50.0,0.0
63816,ZMB,2010,2010,Traditional,1237,60.0,0.0


### 3. Demeaning share by origin

First we show one particular origin

In [39]:
x[(x.iso=='BEN') & (x.year==2013) & (x.bd==1980) & (x.org==1)]

,iso,bd,year,major_religion,org,tot,shr
8937,BEN,1980,2013,Christian,1,5910.0,0.111675
9014,BEN,1980,2013,Muslim,1,23620.0,0.055038
9091,BEN,1980,2013,No Religion,1,3020.0,0.033113
9168,BEN,1980,2013,Other,1,250.0,0.000000
9244,BEN,1980,2013,Traditional,1,2170.0,0.009217


Now we demean. 

- `unweighted` means just taking the migrant share mean within an origin, i.e over `major_religions` within a `iso` X `timevar` X `year` X `org` cell 
- `weighted` means taking a weighted average, where the weight is the sum of `major_religion` person weights

We will use the weighted demeaning as an example as this is the more complicated one

In [40]:
if origin_fe == 'unweighted':
    x['total_org_migshr'] = x.groupby(['iso', timevar, 'year', 'org'])['shr'].transform('mean')
    x['shr'] = x.shr - x.total_org_migshr        
    x.drop('total_org_migshr', axis=1, inplace=True)
elif origin_fe == 'weighted':
    x['tottot'] = x.groupby(['iso', timevar, 'year', 'org'])['tot'].transform('sum')
    x['orgobshr'] = x.tot / x.tottot
    x['wshr'] = x.shr * x.orgobshr
    x['total_org_migshr'] = x.groupby(['iso', timevar, 'year', 'org'])['wshr'].transform('sum')

    # x['aux'] = x.groupby(['iso', timevar, 'year', 'org'])['orgobshr'].transform('sum')
    # print(x[x.major_religion.isin(religs)].aux.min(), x[x.major_religion.isin(religs)].aux.max())
    # x.drop('aux', axis=1)

    x['shr'] = x.shr - x.total_org_migshr
    x.drop(['tottot', 'orgobshr', 'wshr', 'total_org_migshr'], axis=1, inplace=True)    

In [41]:
x[(x.iso=='BEN') & (x.year==2013) & (x.bd==1980) & (x.org==1)]

,iso,bd,year,major_religion,org,tot,shr
8937,BEN,1980,2013,Christian,1,5910.0,0.052196
9014,BEN,1980,2013,Muslim,1,23620.0,-0.004441
9091,BEN,1980,2013,No Religion,1,3020.0,-0.026367
9168,BEN,1980,2013,Other,1,250.0,-0.059480
9244,BEN,1980,2013,Traditional,1,2170.0,-0.050263


### 4. Aggregating up by country X `timevar` X `major_religion`

- We first find the total person weights in a `iso` X timevar X `year` X `major_religion` cell
- Then we divide the number of observations in a `iso` X timevar X `year` X `major_religion` X `origin` cell by that total to get the origin-share of person weights for that `iso` X timevar X `year` X `major_religion` cell
- Then multiply the migrant share by that origin-share to get a weighted migrant share
- Then we sum the weighted migrant shares over all origins in a given `iso` X timevar X `year` X `major_religion` cell
- Finally, we take the average across years (since there may be more than one census year for a country)

In [42]:
x['tottot'] = x.groupby(['iso', timevar, 'year', 'major_religion'])['tot'].transform('sum')
x['orgshr'] = x.tot / x.tottot
x['wshr'] = x.shr * x.orgshr

# x['aux'] = x.groupby(['iso', timevar, 'year', 'major_religion'])['orgshr'].transform('sum')
# print(x[x.major_religion.isin(religs)].aux.min(), x[x.major_religion.isin(religs)].aux.max())
# x.drop('aux', axis=1)

x = x.groupby(['iso', timevar, 'year',  'major_religion'])[['wshr']].sum().reset_index(drop=False)
x = x.groupby(['iso', timevar, 'major_religion'])[['wshr']].mean().reset_index(drop=False)

In [43]:
x

,iso,bd,major_religion,wshr
0,BEN,1890,Christian,0.054740
1,BEN,1890,Muslim,0.044991
2,BEN,1890,No Religion,-0.009505
3,BEN,1890,Other,-0.006521
4,BEN,1890,Traditional,-0.022115
...,...,...,...,...
1060,ZMB,2010,Christian,-0.000457
1061,ZMB,2010,Muslim,0.002391
1062,ZMB,2010,No Religion,0.006697
1063,ZMB,2010,Other,-0.004339


### 5. Finding data availability

Just for visualization, not important

In [44]:
y = x[(x[timevar]>=mint) & (x[timevar]<=maxt)].reset_index(drop=True).copy()
countries_have_data = {}
for relig in religs:
    yi = (y[(y.major_religion==relig)]
           .drop(['major_religion'], axis=1)
           .groupby(['iso', timevar]).sum()
           .unstack()
           )
    yi['wshr'] = np.abs(yi.wshr)
    yi = yi.fillna(0)
    yi[yi>0] = 1
    yi.columns = yi.columns.droplevel(0)
    yi.columns = [f'{c}' for c in yi.columns]
    countries_have_data[relig] = yi.join(iso_df, how='outer').drop('aux', axis=1).fillna(0)

### 6. Merging with population share data

This adds a country's population share for the birth-decade among the countries in the sample

In [45]:
if timevar == 'bd':
    x['bdx'] = x.bd
    x.loc[x.bd<1950, 'bd'] = 1950
    x.loc[x.bd>2000, 'bd'] = 2000
    x = pd.merge(x, pop, on=['iso', 'bd'], how='left')
    x['bd'] = x.bdx
    x.drop('bdx', axis=1, inplace=True)
else:
    x = pd.merge(x, pop[pop.bd==1980].drop('bd', axis=1), on=['iso'], how='left')

In [46]:
x

,iso,bd,major_religion,wshr,popshr
0,BEN,1890,Christian,0.054740,0.018126
1,BEN,1890,Muslim,0.044991,0.018126
2,BEN,1890,No Religion,-0.009505,0.018126
3,BEN,1890,Other,-0.006521,0.018126
4,BEN,1890,Traditional,-0.022115,0.018126
...,...,...,...,...,...
1060,ZMB,2010,Christian,-0.000457,0.031962
1061,ZMB,2010,Muslim,0.002391,0.031962
1062,ZMB,2010,No Religion,0.006697,0.031962
1063,ZMB,2010,Other,-0.004339,0.031962


### 7. Producing overall weighted average

- Normalize the population share within a birth-decade-religion to sum to 1 [since sample of countries not balanced]
- Multiply by migrant shares by population share
- Sum by timevar X `major_religion`

In [47]:
# normalize the population share for each birth decade X major religion
x['aux'] = x.groupby([timevar, 'major_religion'])['popshr'].transform('sum')
x['popshr'] = x.popshr / x.aux
x['wshr'] = x.wshr * x.popshr
x.drop(['iso', 'popshr', 'aux'], axis=1, inplace=True)
x = x.groupby([timevar, 'major_religion']).sum().reset_index(drop=False)
x = x[(x[timevar]>=mint) & (x[timevar]<=maxt)].reset_index(drop=True)